Coueses Recommendation System
----------------------------------------------------
As job tile is exist in course title or course description. The compination of (course title & course description) in one column and apply text vectorization like(TFIDF of FastText) to it and find similarity between courses and jobs.

To recommend courses to job title.

In [1]:
import numpy as np
import pandas as pd
#feature engineering (word vectorization)libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import FastText
#pre_processing libraries
import string
import re
import ast
from nltk.corpus import stopwords
stopwords=stopwords.words('english')
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()

In [2]:
'''read dataset'''
courses_df = pd.read_csv("courses.csv", index_col = 0).drop_duplicates()
courses_df.columns

Index(['_id', 'title', 'url', 'description', 'syllabus', 'skills',
       'ratings_count', 'prerequisites', 'duration', 'category', 'level',
       'schoolName', 'instructors', 'enrolled_students_count', 'avg_rating',
       'num_reviews', 'price', 'source'],
      dtype='object')

In [3]:
'''drop unnessesarry columns'''
courses_df=courses_df.drop(['_id','ratings_count','url',  
                           'duration', 'category', 'level','schoolName', 
                           'instructors', 'enrolled_students_count', 
                           'avg_rating','num_reviews', 'price', 'source'],1)

In [6]:
# clean courses skills
def clean(row):
    row=str(row).lower()
    row = re.sub(r'[^\w]', ' ', row)
    row = re.sub(r'\d+', '',row)
    lemma_text= " ".join(lemma.lemmatize(word) for word in row.split() if word not in stopwords)
    return lemma_text

courses_df['clean_description']= [clean(row) for row in courses_df['description']]
courses_df['clean_title']= [clean(row) for row in courses_df['title']]
courses_df['clean_text']=courses_df['clean_title'] +" "+courses_df['clean_description']  

In [7]:
#will use only tile and clean_text that is compination of (title & description)
#to build the model
courses_df[['title','clean_text']].head(2)

,title,clean_text
0,AWS Certified Solutions Architect - Associate ...,aws certified solution architect associate wan...
1,The Web Developer Bootcamp,web developer bootcamp course need learn web d...


__________________________________________________________________________
Usage of two methods (FastText and TFIDF) to make vectorization to (course title and description)  
to find similarity between it and job title.
Then compare the results of two.

In [8]:
#buid fasttext model
FastTextModel = FastText(courses_df['clean_text'], min_count = 1, size = 40, window = 5)
# apply tfidf to clean text 
vectorizer = TfidfVectorizer()
tfidf_vec = vectorizer.fit_transform(courses_df['clean_text'])

In [21]:
def find_courses(job_title,model_type):
    '''
    input: job title,
           model_type if (1) then it will be fasttext else it will be iftdf
    output:dataframe of recommended courses and similarity value and clean_text        
    
    '''
    job_title=clean(job_title)
    similarity_list=[]
    #n_similarity between job_title and all (course title & describtion) and append it to list
    if model_type==1:
        for i in range(courses_df.shape[0]):
            similarity_list.append(FastTextModel.n_similarity(courses_df['clean_text'].iloc[i], job_title))
    else:
        #transform input_skills by vectorizer
        job_title=vectorizer.transform([job_title])
        for i in range(courses_df.shape[0]):
            similarity_list.append(cosine_similarity(tfidf_vec[i], job_title))
        
    similarity_series = pd.Series(similarity_list).sort_values(ascending=False)
    # getting the indexes of the 10 most similar jobs
    top_10_indexes = list(similarity_series.iloc[0:10].index)
    # Build Dataframe of more similar jobs to inserted skills
    data = ([courses_df.iloc[i,0],similarity_series[i],courses_df.iloc[i,7]] for i in top_10_indexes) 
    df = pd.DataFrame(data , columns = ['Course title','Similarity','clean_text' ])
    return df

In [12]:
#The results of FastText Model
find_courses('Data Scientist',model_type=1)

C:\Users\DINA\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  del sys.path[0]


,Course title,Similarity,clean_text
0,Statistics for Data Science and Business Analysis,0.991212,statistic data science business analysis stati...
1,Statistics for Data Science - Master Data Anal...,0.990946,statistic data science master data analysis da...
2,Statistics Masterclass for Data Science and Da...,0.990691,statistic masterclass data science data analyt...
3,Statistics & Applied Data Science - Business D...,0.990065,statistic applied data science business data a...
4,Introduction to Statistics,0.988506,introduction statistic introductory statistic ...
5,Statistics for Business Analytics and Data Sci...,0.988396,statistic business analytics data science z le...
6,Excel Statistics Essential Training: 1 (2016),0.988049,excel statistic essential training nan
7,Excel Statistics Essential Training: 1,0.988049,excel statistic essential training nan
8,Statistics Made Easy by Example for Analytics/...,0.987772,statistic made easy example analytics data sci...
9,Foundation of Statistics with Minitab,0.987426,foundation statistic minitab introducion descr...


In [23]:
#The results of TFIDF Model
find_courses('Data Scientist',model_type=2)

,Course title,Similarity,clean_text
0,Java for Data Scientists Essential Training,[[0.6578773721842326]],java data scientist essential training nan
1,Introduction To Data Science,[[0.5612131532505049]],introduction data science use r programming la...
2,The Data Scientist’s Toolbox,[[0.5534757775736342]],data scientist toolbox course get introduction...
3,R Programming from Scratch for Data Science- S...,[[0.5484104337274723]],r programming scratch data science step step l...
4,Practical SQL Bootcamp for Data Analysts and D...,[[0.520659407084933]],practical sql bootcamp data analyst data scien...
5,R Programming For Absolute Beginners,[[0.49996539635931453]],r programming absolute beginner learn basic wr...
6,Careers in Data Science A-Z™,[[0.4966539441066395]],career data science z become top level data sc...
7,SQL for Data Science,[[0.4817287632077947]],sql data science data collection increased exp...
8,Want to be a Big Data Scientist?,[[0.46289743993271515]],want big data scientist pursue career data sci...
9,SQL for Data Analysis: Core Data Scientist’s S...,[[0.4447739738255645]],sql data analysis core data scientist skill le...


In [24]:
#The results of FastText Model
find_courses('Full Stack Web Developer',model_type=1)

C:\Users\DINA\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  del sys.path[0]


,Course title,Similarity,clean_text
0,The Full Stack Web Development,0.987466,full stack web development learn full stack we...
1,PowerShell version 5.1 and 6: Step-by-Step,0.985435,powershell version step step exactly need know...
2,JavaScript: Learn Front End Web Development,0.984824,javascript learn front end web development rea...
3,Node with React: Fullstack Web Development,0.984013,node react fullstack web development build dep...
4,Full-Stack Firebase,0.983612,full stack firebase build serverless web apps ...
5,Docker for Developers and DevOps,0.983564,docker developer devops use docker streamline ...
6,The Essential Web Developer Course - Build 12 ...,0.983440,essential web developer course build website a...
7,Become A Full Stack Web Developer - Beginner T...,0.982657,become full stack web developer beginner advan...
8,How To Create A Real Estate Website With Wordp...,0.982311,create real estate website wordpress want crea...
9,Create WordPress Website 2018 with SEO & HTTPS,0.981960,create wordpress website seo http follow step ...


In [25]:
#The results of TFIDF Model
find_courses('Full Stack Web Developer',model_type=2)

,Course title,Similarity,clean_text
0,Become A Full Stack Web Developer - Beginner T...,[[0.8024626633915772]],become full stack web developer beginner advan...
1,The Full Stack Web Development,[[0.7600274615457615]],full stack web development learn full stack we...
2,The Full Stack Web Development Bootcamp 2020,[[0.6606277186831746]],full stack web development bootcamp learn html...
3,Go Full Stack with Spring Boot and React,[[0.6326995493016021]],go full stack spring boot react build first ja...
4,The Complete 2019 Web Development Bootcamp,[[0.5855458542170441]],complete web development bootcamp course need ...
5,Go Full Stack with Spring Boot and Angular,[[0.5844072912676658]],go full stack spring boot angular become full ...
6,The Result-Oriented Web Developer Course - BO...,[[0.5822408258700136]],result oriented web developer course bootcamp ...
7,The Result-Oriented Web Developer Course - BO...,[[0.5822408258700136]],result oriented web developer course bootcamp ...
8,From A Non Programmer To Full Stack .NET Devel...,[[0.5822064701834291]],non programmer full stack net developer full s...
9,Full Stack Web Development Masterclass: Beginn...,[[0.5803077905554829]],full stack web development masterclass beginne...
